In [ ]:
import numpy as np
import pandas as pd
from  matplotlib import pyplot as plt
import os 
from pathlib import Path
import re
import PIL
import cv2
import seaborn as sns 
import copy
import importlib

import spectral_util
importlib.reload(spectral_util)
from spectral_util import *

import fluorescence_util
importlib.reload(fluorescence_util)
from fluorescence_util import *


In [ ]:
# 'board.xlsx'
# 'board_noFlux.xlsx'
# 'flux_20240925_184835.xlsx'
# 'flux-onBoard.xlsx'
# 'lead_noFlux.xlsx'

srcbase = Path("./data/EEM_F-7000_2025-04-11/")
dstdir = Path("./dst/eem/filter")

srcdata = [
    {
        "path": fpath,
        "sample": fpath.stem.split("_")[0],  # 'ABS_20250411' → 'ABS'
        "label": None
    }
    for fpath in srcbase.glob("*.xlsx")
]
srcdata

In [ ]:
for data in srcdata:
    eem = fluorescence_util.EEMF7000(data.get('path'))
    print(eem)

    plt.figure()
    eem.plot_contour(level=100, show_sample_name=True)

In [ ]:
em_bands = eem.em_bands
ex_bands = eem.ex_bands

EEMの1次反射・n次散乱光除去のためのスニペット (クラスメソッドに実装済み)

In [ ]:
verbose = True
_df = copy.deepcopy(eem.eem_df)

def find_nearest(array: np.array, value):
    idx = (np.abs(array - value)).argmin()
    # val = array[idx]
    return int(idx)

def is_out_of_range(value, array: np.array = eem.ex_bands, step=eem.ex_band_step, verbose=False):
    if value+(step/2) < array.min():
        if verbose: print(f"{array.max()} < {value-(step/2)}") 
        return True
    elif array.max() < value-(step/2):
        if verbose: print(f"{array.max()} < {value-(step/2)}")
        return True
    
    return False

def _calc_shift_band(wl_src, degree=1, shift:int=0, band_step=eem.em_band_step):
    ''' eemのemissionの波長からexcitationの1次反射とn次散乱光の波長を計算する
    FIXME 後でGPTに埋めさせる
    :param wl_src: 
    :param degree:
    :param shift:
    :param band_step:

    :returns: 
    '''
    shifted_bands = wl_src * degree+(shift *band_step)

    return shifted_bands

def _elliminate_eem(eem_df, em, step, degree=1, bands_ex:np.array=eem.ex_bands, *, inplace=False, verbose=False):
    '''EEMにおける1次反射と2次散乱光を `np.nan` で埋めて消去する。
    FIXME 後でGPTに埋めさせる
    '''
    if not inplace:
        eem_df = eem_df.copy()

    wl_elim_ex = _calc_shift_band(em, shift= step, degree=degree)
    out_of_range = is_out_of_range(wl_elim_ex, verbose=verbose) 
    if verbose: 
        print(f"base={em} nm:/t({em}, {wl_elim_ex}), out-of-range={out_of_range}")
    idx_elim_ex = find_nearest(bands_ex, wl_elim_ex) # 最も近いexの波長を求める
    wl_elim_ex = bands_ex[idx_elim_ex]
    
    # 最近傍だと範囲を大きく超えていても最大値に張り付く可能性があるので 
    # (600nmの2次光の1200nmでも800nmがexの最大値なので800nmが削除対象に含まれている可能性がある) 
    # 範囲外の波長を除外する

    if em in eem.em_bands and wl_elim_ex in eem.ex_bands and not out_of_range:
        eem_df.loc[em, wl_elim_ex] = np.nan

    return eem_df

def remove_self_reflection_and_scattering_from_eem(eem_df, em_bands=eem.em_bands, margin_steps=3, *, inplace=False, verbose=False):
    if not inplace:
        eem_df = eem_df.copy()

    bands_targets = em_bands
    for target_em in bands_targets:

        for step in range(margin_steps):
            # 1次反射
            eem_df = _elliminate_eem(eem_df, em=target_em, step = -step, degree=1, inplace=inplace, verbose=verbose)
            eem_df = _elliminate_eem(eem_df, em=target_em, step = +step, degree=1, inplace=inplace, verbose=verbose)
            # 2次散乱
            eem_df = _elliminate_eem(eem_df, em=target_em, step = -step, degree=2, inplace=inplace, verbose=verbose)
            eem_df = _elliminate_eem(eem_df, em=target_em, step = +step, degree=2, inplace=inplace, verbose=verbose)
            # 3次散乱
            eem_df = _elliminate_eem(eem_df, em=target_em, step = -step, degree=3, inplace=inplace, verbose=verbose)
            eem_df = _elliminate_eem(eem_df, em=target_em, step = +step, degree=3, inplace=inplace, verbose=verbose)

    return eem_df


In [ ]:
remove_self_reflection_and_scattering_from_eem(eem_df=_df, margin_steps=6, inplace=True)
plt.imshow(_df[eem.ex_bands].values)
# _df.values
_df[eem.ex_bands]

1次反射・n次散乱の除去 (クラスメソッドを使用)

In [ ]:
eem.remove_self_reflection_and_scattering_from_eem(margin_steps=6, inplace=True, )
eem.plot_heatmap()
plt.title(eem.sample)

In [ ]:
for data in srcdata:
    eem = fluorescence_util.EEMF7000(data.get('path'))
    print(eem)

    plt.figure()
    # eem.plot_contour(level=100, show_sample_name=True)

    eem.remove_self_reflection_and_scattering_from_eem(margin_steps=6, inplace=True, )
    eem.plot_heatmap()

    plt.title(eem.sample)



In [ ]:
sample_data = []

for data in srcdata:
    eem = fluorescence_util.EEMF7000(data.get('path'))
    print(eem)

    # plt.figure()
    # eem.plot_contour(level=100, show_sample_name=True)

    eem.remove_self_reflection_and_scattering_from_eem(margin_steps=6, inplace=True, )
    
    eem_matrix = eem.mat
    # eem_df = eem.eem_df
    
    # サンプルごとにnumpy配列に追加
    sample_data.append(eem_matrix)

    print(eem_matrix)

# numpy配列に保存
eem_array = np.array(sample_data)

# [放射波長，励起波長，9種類]の形で保存
np.save('eem_data.npy', eem_array)

# 形状確認
print(f'EEM data shape: {eem_array.shape}')

---

# LED SPD

In [ ]:
_param_set = '200to600nm_narrow'
# _param_set = '200to600nm_wide'
_params = {
    '200to600nm_narrow':{'wl_start': 200, 'wl_end': 600, 'band_steps': 10, 'band_resolution': 5, 'wl_FWHM': 20},
    '200to600nm_wide':  {'wl_start': 200, 'wl_end': 600, 'band_steps': 10, 'band_resolution': 1, 'wl_FWHM': 40},
    }
param = _params[_param_set]

_wl_bands = np.arange(param['wl_start'], param['wl_end'], param['band_steps'])
_df = pd.DataFrame({'wavelength':_wl_bands, })
_df.loc[:,_wl_bands] = 0

for wl in _wl_bands:
    _df.loc[_df.loc[:,'wavelength']==wl,wl] = 1

wl_sensor = _wl_bands


In [ ]:
wl_FWHM = param['wl_FWHM']
var_FEHM = pow(wl_FWHM/2.35, 2)
f"σ={var_FEHM}, for {wl_FWHM} nm of FWHM"

In [ ]:
wl_start = param['wl_start']
wl_end = param['wl_end']
wl_res = param.get('band_resolution',5)
wl = np.array(range(wl_start, wl_end+1, wl_res))
wl_width = wl_end - wl_start
wl_bandstep = param['band_steps']
# wl_peeks = np.array(
#     range(wl_start+round(wl_bandstep/2), wl_end+1, wl_bandstep))
wl_peeks = np.array(range(wl_start, wl_end+1, wl_bandstep))
print("Range: {0}--{1}, ".format(wl_peeks[0],
      wl_peeks[-1]), "Band steps: {}".format(wl_bandstep))

label_wl = '{0}to{1}nm'.format(wl_start, wl_end, )

print(np.array(wl_peeks))
print('# of LEDs: {}'.format(len(wl_peeks)))


## Make SPD

In [ ]:
spds_fill_ = np.array([
    np.exp(-pow(wl - wl_peek_, 2) / var_FEHM)
    for wl_peek_ in wl_peeks
    ])
spds_fill = spds_fill_.T
spds_fill.shape

In [ ]:
fig = plt.figure(1)
ax = fig.add_subplot(1, 1, 1)
plt.plot(
    wl, spds_fill, label=[
    'SPDs'if i == 0 else '_nolegend_' for i in range(spds_fill.shape[1])])
plt.xlim([wl[0], wl[-1]])
plt.ylim([0, plt.ylim()[1]])
plt.xlabel('Wavelength [nm]')
plt.ylabel('Relative Power')
plt.grid(True)
plt.legend(bbox_to_anchor=(1.0, 1.15))


In [ ]:
fig = plt.figure(1)
ax = fig.add_subplot(1, 1, 1)
plt.plot(
    wl, spds_fill[:,7], label=[
    'SPDs'if i == 0 else '_nolegend_' for i in [7,]])
# plt.xlim([wl[0], wl[-1]])
plt.xlim([250, 300])
plt.ylim([0, plt.ylim()[1]])
plt.xlabel('Wavelength [nm]')
plt.ylabel('Relative Power')
plt.grid(True)
plt.legend(bbox_to_anchor=(1.0, 1.15))

spds_fill.shape


----

# EEM × LED

In [ ]:
print(eem_array.shape)  # (10, 81, 81) = (サンプル数（MP），励起，放射)
print(spds_fill.shape)  # (81, 41) = (光強度（各励起波長），LEDの数)

# shapes: (10, 81, 81) @ (81, 41) → (10, 41, 81)
eem_array_clean = np.nan_to_num(eem_array)
fluorescence = np.einsum('sem,el->slm', eem_array_clean, spds_fill)


fluorescence.shape

# eem_array
# fluorescence

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

sample_names = []
for data in srcdata:
    eem = fluorescence_util.EEMF7000(data.get('path'))
    sample_names.append(eem.sample)  # ここでサンプル名を保存

# === 例：LEDの励起波長（列方向） ===
ex_wavelengths = np.linspace(200, 600, spds_fill.shape[0])  # shape: (81,)
led_peak_wavelengths = ex_wavelengths[np.argmax(spds_fill, axis=0)]  # 各LEDのピーク波長を取得（長さ num_leds）

# === 入力：表示したいサンプル番号とピーク波長 ===
sample_name = 'PET'  # 任意の名前を指定
sample_idx = sample_names.index(sample_name)

desired_peak_wavelength = 330  

# === ピーク波長が最も近いLEDを選ぶ ===
led_idx = np.argmin(np.abs(led_peak_wavelengths - desired_peak_wavelength))
print(f"Selected LED {led_idx} with peak wavelength {led_peak_wavelengths[led_idx]:.1f} nm")

# === 放射波長（em） ===
em_wavelengths = np.linspace(200, 600, fluorescence.shape[2])  # shape: (81,)

# === 蛍光スペクトルとLEDのSPDをプロット ===
plt.figure(figsize=(10, 5))

# 蛍光スペクトル
plt.plot(em_wavelengths, fluorescence[sample_idx, led_idx], label=f'Fluorescence (Sample {sample_name})')
plt.xlabel('Emission Wavelength [nm]')
plt.ylabel('Fluorescence Intensity')
plt.grid(True)
plt.legend(loc='upper left')

# 同じ図に励起SPDを重ねる
ax2 = plt.gca().twinx()
ax2.plot(ex_wavelengths, spds_fill[:, led_idx], 'r--', alpha=0.5, label=f'LED SPD (Peak {led_peak_wavelengths[led_idx]:.1f} nm)')
ax2.set_ylabel('LED Excitation SPD')
ax2.legend(loc='upper right')

plt.title(f'Sample {sample_name}, LED with Peak {led_peak_wavelengths[led_idx]:.1f} nm')
plt.tight_layout()

# plt.xlim(310, 380)
plt.show()


In [ ]:
# 蛍光成分以外がnanで保存されているか
eem_arrasample_idx = 0
eem_matrix = eem_array[sample_idx]  # shape: (ex, em) = (81, 81)

plt.figure(figsize=(6, 5))
plt.imshow(eem_matrix, origin='lower', cmap='jet', aspect='auto')
plt.colorbar(label='Intensity')
plt.title(f'EEM Matrix for Sample {sample_names[sample_idx]} (before NaN to 0)')
plt.xlabel('Excitation index')
plt.ylabel('Emission index')
plt.plot(np.arange(81), np.arange(81), 'w--')  # 対角線（自己波長）を強調
plt.show()

In [ ]:
# 励起波長（デルタ関数，理想的なナローバンド）ごとの放射波長plot 
import numpy as np
import matplotlib.pyplot as plt

# 波長軸
ex_wavelengths = np.linspace(200, 600, eem_array.shape[1])
em_wavelengths = np.linspace(200, 600, eem_array.shape[2])

# 指定：サンプル名と励起波長のリスト
sample_name = "PET"
desired_ex_wavelengths = [310
                          , 320
                          , 330
                          , 340
                          , 350
                          ]  # 任意の励起波長（複数）

# インデックス変換
sample_idx = sample_names.index(sample_name)
ex_indices = [np.argmin(np.abs(ex_wavelengths - wl)) for wl in desired_ex_wavelengths]

# プロット
plt.figure(figsize=(10, 5))
for i, ex_idx in enumerate(ex_indices):
    em_spectrum = eem_array_clean[sample_idx, ex_idx, :]
    plt.plot(em_wavelengths, em_spectrum, label=f'Ex={ex_wavelengths[ex_idx]:.1f} nm')

plt.xlabel('Emission Wavelength [nm]')
plt.ylabel('Fluorescence Intensity')
plt.title(f'Sample: {sample_name} - Fluorescence for Various Excitations')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
